In [1]:
print("ekans")

ekans


In [15]:
import onlinehd as Onlinehd
import warnings

from time import time

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.preprocessing
import sklearn.model_selection
import torch
import Config
import math

In [4]:
datasetname="Microsoft Challenge BIG 2015"
datasets=[
    "KDD Cup 1999",                            #0
    "Microsoft Challenge BIG 2015"             #1
]
presets = {
    "KDD Cup 1999": {
        "NeuralHD": [300,2,3,.1],
        "OnlineHD": [300,1.0,.1,30,True],
        "MLP": [100,5,.001],
        "SVM": [10000]
    },
    "Microsoft Challenge BIG 2015": {
        "NeuralHD": [3000,6,10,.1],
        "OnlineHD": [3000,1.0,.1,30,True],
        "MLP": [100,30,.001],
        "SVM": [None]
    }
}
def normalized(x,y):
    xtrain, x_test, ytrain, y_test = None,None,None,None
    x, x_test, y, y_test = sklearn.model_selection.train_test_split(x, y, shuffle=True)
    scaler = sklearn.preprocessing.Normalizer().fit(x)
    x = scaler.transform(x)
    x_test = scaler.transform(x_test)

    # changes data to pytorch's tensors
    x = torch.from_numpy(x).float()
    y = torch.from_numpy(y).long()
    x_test = torch.from_numpy(x_test).float()
    y_test = torch.from_numpy(y_test).long()
    return x.numpy(), x_test.numpy(), y.numpy(), y_test.numpy(), scaler
def getuniquevalues(columnname,df):
    values={}
    i=0
    for entry in df[columnname]:
        if entry not in values:
            values[entry]=i
            i+=1
    return values
def get_dataset(name):
    if name==datasets[0]:
        path="../../Data/"
        attacks_types = {
            'normal': 'normal','back': 'dos','buffer_overflow': 'u2r','ftp_write': 'r2l','guess_passwd': 'r2l',
        'imap': 'r2l','ipsweep': 'probe','land': 'dos','loadmodule': 'u2r','multihop': 'r2l','neptune': 'dos',
        'nmap': 'probe','perl': 'u2r','phf': 'r2l','pod': 'dos','portsweep': 'probe','rootkit': 'u2r','satan': 'probe',
        'smurf': 'dos','spy': 'r2l','teardrop': 'dos','warezclient': 'r2l','warezmaster': 'r2l',
        }
        cols ="""duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,
        urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,
        num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,
        count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,
        diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,
        dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,
        dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate"""
        
        columns =[]
        for c in cols.split(','):
            if(c.strip()):
                columns.append(c.strip())
        print(len(columns))
        columns.append('target')
        print(len(columns))

        attack_categories=["dos","u2r","r2l",'probe','normal']
        df = pd.read_csv(path+"kddcup.data_10_percent.gz", names = columns)
        df['Attack Type'] = df.target.apply(lambda r:attacks_types[r[:-1]])
        del df['target']
        df.head()
        num_cols = df._get_numeric_data().columns
        
        cate_cols = list(set(df.columns)-set(num_cols))
        cate_cols.remove('Attack Type')
        for col in cate_cols:
            df[col]=df[col].map(getuniquevalues(col,df))
        data=df.to_numpy()
        Y=df['Attack Type'].map(getuniquevalues('Attack Type',df))
        Y=Y.to_numpy()
        X=data[:,:-1]
        print(Y.shape)
        print(X.shape)
        print(getuniquevalues('Attack Type',df))
        xtrain, x_test, ytrain, y_test,scaler= normalized(X,Y)
    if name==datasets[1]:
        path="../../Data/malware-classification/"
        map={}
        mapping=pd.read_csv(path + "trainLabels.csv")
        Y=mapping["Class"].to_numpy()
        for i in range(0,len(Y)):
            map[mapping["Id"][i]]=mapping["Class"][i]-1
        byte_features=pd.read_csv(path+"result.csv")
        byte_features['ID']  = byte_features['ID'].str.split('.').str[0]
        byte_features.head(3)
        byte_features['ID']=byte_features['ID'].map(map)
        data=byte_features.to_numpy()
        X=data[:,1:]
        Y=data[:,0]
        xtrain, x_test, ytrain, y_test,scaler= normalized(X,Y)
    return xtrain,x_test,ytrain,y_test

xtrain,x_test,ytrain,y_test=get_dataset(datasetname)

In [5]:
xtraintorch=torch.from_numpy(xtrain)
ytraintorch=torch.from_numpy(ytrain)
x_testtorch=torch.from_numpy(x_test)
y_testtorch=torch.from_numpy(y_test)

In [16]:
def get_class_accuracy_breakdown(model,x_test,y_test, output=False):
    acc=[]
    points=[]
    try:
        for i in range (0,len(np.unique(y_test))):
            yhat= model.predict(x_test[y_test==i])
            if len(yhat.shape)==2:
                yhat=np.array([row.argmax() for row in yhat])
            acc.append((yhat==i).mean())
            points.append(len(yhat))
    except:
        for i in range (0,len(np.unique(y_test))):
            yhat= model.predict(torch.from_numpy(x_test[y_test==i]))
            acc.append((yhat==i).float().mean())
            points.append(len(yhat))
    # print(yhat[:30])
    totacc=sum([acc[i]*points[i] for i in range(0,len(acc))])/sum(points)
    if output:
        plt.bar(range(0,len(acc)),acc,color=np.random.rand(3,))
        plt.title("Accuracy Total: " + str(totacc))
        plt.show()
    return totacc

In [28]:
#change
def cos_cdist(x1 : torch.Tensor, x2 : torch.Tensor, eps : float = 1e-8):
    #Cosine Similarity
    eps = torch.tensor(eps, device=x1.device)
    norms1 = x1.norm(dim=1).unsqueeze_(1).max(eps)
    norms2 = x2.norm(dim=1).unsqueeze_(0).max(eps)
    cdist = x1 @ x2.T
    cdist.div_(norms1).div_(norms2)
    return cdist
class NeuralHD:
    def __init__(self, classes : int, features : int, dim : int = 400, batch_size=1,trainopt=3,bestinclass=False,lr=.003):
        #Configure for hdb, hdc, and hde classes
        print("test")
        self.param=Config.config
        self.param['nClasses'] = classes
        self.param['nFeatures']= features
        #hypervector size
        self.param['D']=dim
        self.param['lr']=.0001
        self.batch_size=batch_size
        self.base = torch.empty(self.param['D']).uniform_(0.0, 2*math.pi)

        self.bestinclass=bestinclass
        #encoder
        self.hde=None
        #classifier
        self.hdc=None
        # Initialize basis in gaussian distribution
        self.basis = torch.normal(0,1,size=(self.param["D"],self.param["nFeatures"]))
        # Initialize classification hypervectors
        self.classes = torch.zeros((self.param['nClasses'], self.param['D']))
        self.prevacc=0
        self.trainoption=trainopt
        self.trainfunctions=[self.train,self.train2,self.train3]
        # self.param['lr']=.1
        # self.hdc = HD_classifier(self.param["D"], self.param["nClasses"], 0)
        self.trainaccuracies=[]
        self.testaccuracies=[]
        self.medians=[]
    def __call__(self, x : torch.Tensor):
        #return predicted values
        return self.predict(x)
    def encode(self,x):
        n = x.size(0)
        bsize = min([x.size(1),1024])
        h = torch.empty(n, self.basis.shape[0], device=x.device, dtype=x.dtype)
        temp = torch.empty(bsize, self.basis.shape[0], device=x.device, dtype=x.dtype)

        # we need batches to remove memory usage
        for i in range(0, n, bsize):
            torch.matmul(x[i:i+bsize], self.basis.T, out=temp)

            # self.noise ... I haven't seen any indication that it works better 
            # if self.noise:
            torch.add(temp, self.base, out=h[i:i+bsize])#h[i:i+bsize]=temp# torch.add(temp, self.base, out=h[i:i+bsize])
            # else:
            # h[i:i+bsize]=temp

            h[i:i+bsize].cos_()#.mul_(temp.sin_())
        # print(h.shape)
        return h
    def train(self,h,y):
        print("1")
        # r=torch.randperm(y.size(0))
        # y=y[r]
        # h=h[r,:]
        n = h.size(0)
        batch_size = min([y.size(0), self.batch_size])#64
        for i in range(0, n, batch_size):
            h_ = h[i:i+batch_size]
            y_ = y[i:i+batch_size]
            scores = cos_cdist(h_, self.classes)#cos
            y_pred = scores.argmax(1)
            wrong = y_ != y_pred

            # computes alphas to update model
            # alpha1 = 1 - delta[lbl] -- the true label coefs
            # alpha2 = delta[max] - 1 -- the prediction coefs
            aranged = torch.arange(h_.size(0), device=h_.device)
            alpha1 = (1.0 - scores[aranged,y_]).unsqueeze_(1)
            alpha2 = (scores[aranged,y_pred] - 1.0).unsqueeze_(1)

            for lbl in y_.unique():
                m1 = wrong & (y_ == lbl) # mask of missed true lbl
                m2 = wrong & (y_pred == lbl) # mask of wrong preds
                self.classes[lbl] += self.param['lr']*(alpha1[m1]*h_[m1]).sum(0)
                self.classes[lbl] += self.param['lr']*(alpha2[m2]*h_[m2]).sum(0)
            # if self.test(h,y)<self.prevacc:
            #     for lbl in y_.unique():
            #         m1 = wrong & (y_ == lbl) # mask of missed true lbl
            #         m2 = wrong & (y_pred == lbl) # mask of wrong preds
            #         self.classes[lbl] -= self.param['lr']*(alpha1[m1]*h_[m1]).sum(0)
            #         self.classes[lbl] -= self.param['lr']*(alpha2[m2]*h_[m2]).sum(0)
            # else:
            #     self.prevacc=self.test(h,y)
    def train2(self,h,y):
        # def fit(self, data, label, param = None):
        print("2")
        assert self.param["D"] == h.size(1)
        #if self.first_fit:
        #    sys.stderr.write("Fitting with configuration: %s \n" % str([(k,param[k]) for k in self.options]))

        # Actual fitting

        # handling dropout

        # fit
        r = torch.randperm(h.shape[0])
        correct = 0
        count = 0
        for i in r:
            sample = h[i] 
            answer = y[i]
            #maxVal = -1
            #guess = -1
            #for m in range(self.nClasses):
            #    val = kernel(self.classes[m], sample)
            #    if val > maxVal:
            #        maxVal = val
            #        guess = m
            vals = cos_cdist(sample.unsqueeze(1).T, self.classes)
            # print(vals)
            guess = torch.argmax(vals)
            if guess != answer:
                self.classes[guess]-=self.param['lr']*h[i]*(1-vals[0,guess])
                self.classes[answer]+=self.param['lr']*h[i]*(1-vals[0,answer])
                # acc=self.test2(h[r][:100],y)
                # if acc<=self.prevacc:
                #     self.classes[guess]+=self.param['lr']*h[i]
                #     self.classes[answer]-=self.param['lr']*h[i]
                # else:
                #     self.prevacc=acc
            else:
                correct += 1
            count += 1
        return correct / count
    
    def train3(self,h,y):
        # def fit(self, data, label, param = None):
        print("3")
        assert self.param["D"] == h.size(1)
        #if self.first_fit:
        #    sys.stderr.write("Fitting with configuration: %s \n" % str([(k,param[k]) for k in self.options]))

        # Actual fitting

        # handling dropout

        # fit
        r = torch.randperm(y.size(0))
        y=y[r]
        h=h[r,:]
        correct = 0
        count = 0
        for i in range(0,y.size(0),self.batch_size):
            sample = h[i:i+self.batch_size] 
            answers = y[i:i+self.batch_size]
            #maxVal = -1
            #guess = -1
            #for m in range(self.nClasses):
            #    val = kernel(self.classes[m], sample)
            #    if val > maxVal:
            #        maxVal = val
            #        guess = m
            vals = cos_cdist(sample, self.classes)
            # print(vals)
            guesses = vals.argmax(1)
            # print(guesses)
            for j in range(0,answers.size(0)):
                if guesses[j] != answers[j]:
                    # print(answers[j])
                    self.classes[guesses[j]]-=self.param['lr']*h[i+j]*(1-vals[0,guesses[j]])
                    self.classes[answers[j]]+=self.param['lr']*h[i+j]*(1-vals[0,answers[j]])
                    # acc=self.test2(h[r][:100],y)
                    # if acc<=self.prevacc:
                    #     self.classes[guess]+=self.param['lr']*h[i]
                    #     self.classes[answer]-=self.param['lr']*h[i]
                    # else:
                    #     self.prevacc=acc
                else:
                    correct += 1
                count += 1
        return correct / count

    def predict(self,x):
        #return predictions based on similarity of encoded inputs to classification hypervectors
        return  cos_cdist(self.encode(x), self.classes).argmax(1)
    def fit(self,traindata, trainlabels,
                   epochs,
                   regenloops,  # list of effective dimensions to reach 
                   fractionToDrop # drop/regen rate 
                    ):
        # find encoded training vectors

        # calculate amount of dropped dimensions based on percent and original dimension
        amountDrop = int(fractionToDrop * self.param['D'])#self.param.D?
        # print("Updating times:", regenloops)

        for i in range(regenloops+1): # For each eDs to reach, will checkpoints
            # compute new encoded data
            trainencoded = self.encode(traindata)
            testencoded = self.encode(x_testtorch)
            
            # print("regenloop: " + str(i))
            # train for specified number of epochs
            # Do the train 
            self.prevacc=0
            iterscorestrain=[]
            iterscorestest=[]
            maxval=0
            temp=None
            for j in range(epochs):
                # do one pass of training
                # print(self.classes[:,8])
                result=self.trainfunctions[self.trainoption](trainencoded, trainlabels)
                trainaccuracy= self.test(trainencoded,trainlabels)
                testaccuracy= self.test(testencoded,y_testtorch)
                print(testaccuracy)
                iterscorestrain.append(trainaccuracy)
                iterscorestest.append(testaccuracy)

                if self.bestinclass and trainaccuracy>maxval:
                    temp=copy.deepcopy(self.classes)
                    maxval=trainaccuracy
                    print(testaccuracy)
                # print(j)
            if self.bestinclass:
                self.classes=temp
            
            self.trainaccuracies+=iterscorestrain
            self.testaccuracies+=iterscorestest
            self.medians.append(np.median(np.array(iterscorestrain)))
                # print(self.prevacc)
            #if its the last regeneration training, stop before doing another dimension drop; stop if 100% accuracy
            if i==regenloops:
                return #self.hdc,self.hde - unnecessary now that hdc and hde are within a class
            # print("regen" +str(i))
            #do the dimension drop and regeneration
            normed_classes = torch.nn.functional.normalize(self.classes)
            #calculate variances for each dimension
            var = torch.var(normed_classes, 0) 
            assert len(var) == self.param['D']
            # rank each entry in variances from smallest to largest
            order = torch.argsort(var)
            #drop amountDrop bases
            toDrop = order[:amountDrop]
            #            ----------------
            #attempted reverse drop
            # if amountDrop<0:
            #     toDrop = order[-amountDrop:]
            #            ----------------
            #Update basis
            #For each dimension designated to be dropped
            for i in toDrop:
                #generate a new ith vector in the basis
                self.basis[i] = torch.normal(self.param["mu"],self.param["sigma"], size=(self.param['nFeatures'],))
            #Update Classes
            #            --------------
            #This code was left out. Maybe useful?
            for i in toDrop:
                self.classes[:,i] = torch.zeros(self.param['nClasses'])
            #            --------------

            self.classes=torch.nn.functional.normalize(self.classes)
            # self.batch_size=int(np.ceil(self.batch_size/2))
            # if self.batch_size==1:
            #     self.param['lr']=self.param['lr']/2
        return "error","error"
    def _iterative_fit(self, h, y, lr, epochs):
        h=self.encode(h)
        n = h.size(0)
        for epoch in range(epochs):
            for i in range(0, n, self.batch_size):
                h_ = h[i:i+self.batch_size]
                y_ = y[i:i+self.batch_size]
                scores = cos_cdist(h_, self.classes)
                y_pred = scores.argmax(1)
                wrong = y_ != y_pred

                # computes alphas to update model
                # alpha1 = 1 - delta[lbl] -- the true label coefs
                # alpha2 = delta[max] - 1 -- the prediction coefs
                aranged = torch.arange(h_.size(0), device=h_.device)
                alpha1 = (1.0 - scores[aranged,y_]).unsqueeze_(1)
                alpha2 = (scores[aranged,y_pred] - 1.0).unsqueeze_(1)

                for lbl in y_.unique():
                    m1 = wrong & (y_ == lbl) # mask of missed true lbl
                    m2 = wrong & (y_pred == lbl) # mask of wrong preds
                    self.classes[lbl] += lr*(alpha1[m1]*h_[m1]).sum(0)
                    self.classes[lbl] += lr*(alpha2[m2]*h_[m2]).sum(0)
    def test(self,x_encoded, y_labels):
            yhat= cos_cdist(x_encoded, self.classes).argmax(1)
            return (yhat==y_labels).float().mean()
    def test2(self,x_encoded,y_labels):
        yhat=torch.zeros(y_labels.size(0))
        i=0
        for v in x_encoded:
            sims=torch.matmul(v,self.classes.T)
            yhat[i]=torch.argmax(sims)
            i+=1
        return (yhat==y_labels).float().mean()

In [32]:
model=NeuralHD(len(np.unique(y_test)),xtrain.shape[1],1024,batch_size=1,trainopt=2,bestinclass=False)
model.fit(xtraintorch,ytraintorch,15,4,.1)

test
3


<ipython-input-28-b2219913e319>:51: UserWarning: An output with one or more elements was resized since it had shape [257, 1024], which does not match the required output shape [184, 1024].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  torch.matmul(x[i:i+bsize], self.basis.T, out=temp)
<ipython-input-28-b2219913e319>:51: UserWarning: An output with one or more elements was resized since it had shape [257, 1024], which does not match the required output shape [147, 1024].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cp

tensor(0.8616)
3
tensor(0.8874)
3
tensor(0.9073)
3
tensor(0.8495)
3
tensor(0.9006)
3
tensor(0.9003)
3
tensor(0.9080)
3
tensor(0.9139)
3
tensor(0.8877)
3
tensor(0.9253)
3
tensor(0.9323)
3
tensor(0.8947)
3
tensor(0.9238)
3
tensor(0.9404)
3
tensor(0.9187)
3
tensor(0.9507)
3
tensor(0.9503)
3
tensor(0.9551)
3
tensor(0.9507)
3
tensor(0.9525)
3
tensor(0.9525)
3
tensor(0.9547)
3
tensor(0.9573)
3
tensor(0.9544)
3
tensor(0.9555)
3
tensor(0.9544)
3
tensor(0.9529)
3
tensor(0.9547)
3
tensor(0.9499)
3
tensor(0.9525)
3
tensor(0.9551)
3
tensor(0.9566)
3
tensor(0.9544)
3
tensor(0.9566)
3
tensor(0.9566)
3
tensor(0.9566)
3
tensor(0.9544)
3
tensor(0.9492)
3
tensor(0.9573)
3
tensor(0.9551)
3
tensor(0.9547)
3
tensor(0.9551)
3
tensor(0.9540)
3
tensor(0.9577)
3
tensor(0.9562)
3
tensor(0.9569)
3
tensor(0.9562)
3
tensor(0.9551)
3
tensor(0.9558)
3
tensor(0.9577)
3
tensor(0.9562)
3
tensor(0.9514)
3
tensor(0.9577)
3
tensor(0.9551)
3
tensor(0.9569)
3
tensor(0.9577)
3
tensor(0.9558)
3
tensor(0.9580)
3
tensor(0.9566)